In [2]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import nltk
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import codecs
from tqdm import tqdm
import tensorflow as tf
import shutil

from keras.models import Sequential
from keras.layers import LSTM, GRU,SimpleRNN
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.layers import BatchNormalization
from keras.utils import to_categorical
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from imblearn.over_sampling import SMOTE
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import vectorize_functions
import resampler


import warnings
warnings.filterwarnings("ignore")

In [6]:
train_cleaned_data = pd.read_csv("..\\..\\..\\data\\twitter_hate-speech\\train_basic_cleaned.csv")
test_cleaned_data = pd.read_csv("..\\..\\..\\data\\twitter_hate-speech\\test_basic_cleaned.csv")

X_train_data = train_cleaned_data['tweet']
y_train_data = train_cleaned_data['label']

X_test_data = test_cleaned_data['tweet']



In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_data)
word_index = tokenizer.word_index

xtrain_seq = tokenizer.texts_to_sequences(X_train_data)
xtest_seq = tokenizer.texts_to_sequences(X_test_data)

max_len = max(len(seq) for seq in xtrain_seq) 
xtrain_pad = pad_sequences(xtrain_seq, padding='post', maxlen=max_len)
xtest_pad = pad_sequences(xtest_seq, padding='post', maxlen=max_len)


In [33]:
print('Preprocessed text', X_train_data[1])
print('\nTokenized text', xtrain_seq[1])
print('\nPadded text', xtrain_pad[1])
print('\nPadded text Length: ', len(xtrain_pad[1]))

Preprocessed text @user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked

Tokenized text [1, 1, 165, 9, 5485, 2484, 5, 64, 490, 638, 65, 70, 1485, 7896, 10268, 8, 7897, 16187, 10269]

Padded text [    1     1   165     9  5485  2484     5    64   490   638    65    70
  1485  7896 10268     8  7897 16187 10269     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]

Padded text Length:  42


In [8]:
X_train, X_val, y_train, y_val = train_test_split(xtrain_pad, y_train_data, test_size=0.2, random_state=42)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

### Embeddings

In [9]:
def load_embeddings(file_path, embedding_dim, word_index):
    embedding_index = {}
    with open(file_path, encoding='utf-8') as f:
        for line in f:
            values = line.rstrip().split(' ')
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embedding_index[word] = coefs

    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return embedding_index, embedding_matrix


In [29]:
import requests, zipfile, io
zip_file_url = "http://nlp.stanford.edu/data/glove.6B.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [30]:
import requests, zipfile, io
zip_file_url = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

### Models

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report

def train_rnn(x_train, y_train, x_val, y_val, word_index, max_len, embedding_matrix=None, embedding_dim=300, epochs=5, batch_size=64):
    model = Sequential()
    if embedding_matrix is not None:
        model.add(Embedding(len(word_index) + 1, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
    else:
        model.add(Embedding(len(word_index) + 1, embedding_dim, input_length=max_len))
    model.add(SimpleRNN(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))
    return model, history

def train_lstm(x_train, y_train, x_val, y_val, word_index, max_len, embedding_matrix=None, embedding_dim=300, epochs=5, batch_size=64):
    model = Sequential()
    if embedding_matrix is not None:
        model.add(Embedding(len(word_index) + 1, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
    else:
        model.add(Embedding(len(word_index) + 1, embedding_dim, input_length=max_len))
    model.add(LSTM(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))
    return model, history

def train_gru(x_train, y_train, x_val, y_val, word_index, max_len, embedding_matrix=None, embedding_dim=300, epochs=5, batch_size=64):
    model = Sequential()
    if embedding_matrix is not None:
        model.add(Embedding(len(word_index) + 1, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
    else:
        model.add(Embedding(len(word_index) + 1, embedding_dim, input_length=max_len))
    model.add(SpatialDropout1D(0.3))
    model.add(GRU(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))
    return model, history


### Training

In [ ]:
glove_file_path = 'glove.6B.300d.txt'
fasttext_file_path = 'wiki-news-300d-1M.vec'
embedding_dim = 300
glove_embedding_index, glove_embedding_matrix = load_embeddings(glove_file_path, embedding_dim, word_index)
fasttext_embedding_index, fasttext_embedding_matrix = load_embeddings(fasttext_file_path, embedding_dim, word_index)

##### LSTM

In [ ]:
print("Training LSTM with GloVe Embeddings")
lstm_glove, lstm_glove_history = train_lstm(
    X_train_resampled, y_train_resampled, X_val, y_val, 
    word_index, max_len, embedding_matrix=glove_embedding_matrix)

print("Training LSTM with FastText Embeddings")
lstm_fasttext, lstm_fasttext_history = train_lstm(
    X_train_resampled, y_train_resampled, X_val, y_val, 
    word_index, max_len, embedding_matrix=fasttext_embedding_matrix)

print("Training LSTM with Word2Vec Embeddings")
lstm_word2vec, lstm_word2vec_history = train_lstm(
    X_train_resampled, y_train_resampled, X_val, y_val, 
    word_index, max_len)


##### GRU

In [ ]:
print("Training GRU with GloVe Embeddings")
gru_glove, gru_glove_history = train_gru(
    X_train_resampled, y_train_resampled, X_val, y_val, 
    word_index, max_len, embedding_matrix=glove_embedding_matrix)

print("Training GRU with FastText Embeddings")
gru_fasttext, gru_fasttext_history = train_gru(
    X_train_resampled, y_train_resampled, X_val, y_val, 
    word_index, max_len, embedding_matrix=fasttext_embedding_matrix)

print("Training GRU with Word2Vec Embeddings")
gru_word2vec, gru_word2vec_history = train_gru(
    X_train_resampled, y_train_resampled, X_val, y_val, 
    word_index, max_len)


### Evaluate

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix

def evaluate_model(model, history, X_val, y_val, X_test=None, test_data=None, output_file=None):
    """
    Evaluates the model performance on validation data and predicts labels on test data (if provided).
    
    Parameters:
    - model: Trained model to evaluate.
    - history: Training history object for visualization of metrics.
    - X_val: Validation data (features).
    - y_val: Validation data (labels).
    - X_test: Test data (features, optional, for prediction).
    - test_data: Original test data (optional, for saving predictions with original tweets).
    - output_file: File path to save test predictions (optional).
    
    Returns:
    None
    """
    # Step 1: Plot Training and Validation Accuracy and Loss
    plt.figure(figsize=(12, 6))
    
    # Accuracy Plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Loss Plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Step 2: Evaluate on Validation Data
    print("\nEvaluating on Validation Data...")
    y_val_pred = (model.predict(X_val) > 0.5).astype(int)
    
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    f1 = f1_score(y_val, y_val_pred)
    roc_auc = roc_auc_score(y_val, model.predict(X_val))
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    
    print("\nClassification Report:")
    print(classification_report(y_val, y_val_pred))
    
    # Confusion Matrix
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['No Hate Speech', 'Hate Speech'], yticklabels=['No Hate Speech', 'Hate Speech'])
    plt.title('Confusion Matrix - Validation Data')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()
    
    # Step 3: Predict on Test Data (if provided)
    if X_test is not None:
        print("\nPredicting on Test Data...")
        test_predictions = model.predict(X_test)
        predicted_labels = (test_predictions > 0.5).astype(int)
        
        # Save predictions to CSV (if test_data and output_file are provided)
        if test_data is not None and output_file is not None:
            results = pd.DataFrame({
                'tweet': test_data['tweet'],
                'predicted_label': predicted_labels.flatten()
            })
            results.to_csv(output_file, index=False)
            print(f"Test predictions saved to {output_file}")
        else:
            print("Test predictions are ready (not saved).")
        
        return predicted_labels

    return None


#### LSTM

In [ ]:

evaluate_model(
    gru_glove, gru_glove_history, X_val, y_val, 
    X_test=xtest_pad, test_data=test_cleaned_data, output_file="gru_glove_predictions.csv"
)

evaluate_model(
    gru_fasttext, gru_fasttext_history, X_val, y_val, 
    X_test=xtest_pad, test_data=test_cleaned_data, output_file="gru_fasttext_predictions.csv"
)

evaluate_model(
    gru_word2vec, gru_word2vec_history, X_val, y_val, 
    X_test=xtest_pad, test_data=test_cleaned_data, output_file="gru_word2vec_predictions.csv"
)

#### GRU

In [ ]:
evaluate_model(
    lstm_glove, lstm_glove_history, X_val, y_val, 
    X_test=xtest_pad, test_data=test_cleaned_data, output_file="lstm_glove_predictions.csv"
)


evaluate_model(
    lstm_fasttext, lstm_fasttext_history, X_val, y_val, 
    X_test=xtest_pad, test_data=test_cleaned_data, output_file="lstm_fasttext_predictions.csv"
)

evaluate_model(
    lstm_word2vec, lstm_word2vec_history, X_val, y_val, 
    X_test=xtest_pad, test_data=test_cleaned_data, output_file="lstm_word2vec_predictions.csv"
)
